# Part 4: Using GPU acceleration with PyTorch

In [0]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

    100% |████████████████████████████████| 92kB 3.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/6c/cb/69/466aef9cee879fb8f645bd602e34d45e754fb3dee2cb1a877a
Successfully built torchbearer


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [0]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[ 0.8004, -0.1185,  0.0766,  ..., -0.2498, -0.9295, -0.8177],
        [ 0.8889,  0.0142,  0.3281,  ...,  0.7931,  2.1969, -0.3727],
        [ 1.1194, -2.1351,  0.5316,  ..., -2.3084, -0.4073,  0.1930],
        ...,
        [ 0.2112,  0.7200, -0.4899,  ..., -0.4995,  0.1286, -0.9344],
        [-0.2539, -0.7855,  0.0216,  ..., -0.2597, -1.5844, -0.4841],
        [-1.3071,  0.7740, -0.1943,  ...,  1.1912,  0.2710,  0.2342]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [0]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[-1.8405e+00, -3.5815e-03, -1.1253e+00,  ..., -1.2303e+00,
          4.2462e-02, -2.5102e-01],
        [ 8.4235e-01,  7.2835e-02,  3.3401e-02,  ...,  1.7723e+00,
          4.4109e-01, -2.4714e-01],
        [ 5.5433e-01,  1.0361e+00, -1.3867e+00,  ...,  2.5300e+00,
          1.9590e-01, -1.2068e+00],
        ...,
        [ 1.8813e-01,  6.6440e-02, -3.9482e-01,  ..., -1.2430e+00,
         -5.5013e-01,  9.9872e-04],
        [-7.5560e-01,  1.4115e+00, -3.7997e-01,  ...,  6.1002e-01,
          6.8250e-01,  1.0520e+00],
        [ 1.2518e-02, -3.1877e-01, -9.7710e-02,  ...,  5.8166e-01,
          3.5808e-01,  1.9888e-01]], device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [0]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-0.8771,  0.3147, -0.2082,  ..., -0.0903, -0.2134, -0.0570],
        [ 0.2190, -0.2789,  0.5547,  ..., -1.1738,  0.3791,  0.1029],
        [-0.3435, -0.4106,  0.4151,  ..., -1.9121, -0.5293, -0.6691],
        ...,
        [-0.8667,  0.0894,  1.6767,  ..., -0.0143,  0.2878,  0.0705],
        [ 2.8974, -0.0853, -0.3223,  ..., -0.1672,  0.3421,  0.0757],
        [ 0.9058,  0.7595,  0.0652,  ...,  1.5239, -0.1545,  0.4137]],
       device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

**My answer:**
A GPU has thousands of cores, while a CPU usually has no more than 12 cores, so GPU has powerful parallel processing power.

CNN and deep learning involve huge amount of matrix multiplications and other operations which can be massively parallelized, so GPU perform computations faster.

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

**My answer:**
It should be the DRAM limits of GPUs. 
As the development of deep learning, deeper and deeper networks are generated, which means much more memory are needed.